In [32]:
import utils.Utils as ut
import os
import pandas as pd

from entity.blockchain.Address import Address
from utils.DataLoader import DataLoader
from utils.Settings import Setting
from utils.ProjectPath import ProjectPath
from collections import Counter
from utils import ChartDrawer as cd

path = ProjectPath()
setting = Setting()

uniswap_tokens = pd.read_csv(os.path.join(path.panv2_processed_path, "token_info.csv"))
uniswap_pools = pd.read_csv(os.path.join(path.panv2_processed_path, "pool_info.csv"))
display(uniswap_tokens.head())
display(uniswap_pools.head())

ENVIRONMENT VARIABLES
DATA PATH: /mnt/Storage/Data/Blockchain/DEX


,token,name,symbol,decimals,totalSupply
0,0x0E09FaBB73Bd3Ade0a17ECC321fD13a19e81cE82,PancakeSwap Token,Cake,18,1808213379751051201610995531
1,0xe9e7CEA3DedcA5984780Bafc599bD69ADd087D56,BUSD Token,BUSD,18,365596245875689184460959985
2,0xbb4CdB9CBd36B01bD1cBaEBF2De08d9173bc095c,Wrapped BNB,WBNB,18,1356006817363359720577984
3,0x3EE2200Efb3400fAbB9AacF31297cBdD1d435D47,Cardano Token,ADA,18,451907177779999999990000000
4,0xAD6cAEb32CD2c308980a548bD0Bc5AA4306c6c18,Band Protocol Token,BAND,18,2300000000000000000000000


,pool,token0,token1
0,0x804678fa97d91B974ec2af3c843270886528a9E6,0x0E09FaBB73Bd3Ade0a17ECC321fD13a19e81cE82,0xe9e7CEA3DedcA5984780Bafc599bD69ADd087D56
1,0x0eD7e52944161450477ee417DE9Cd3a859b14fD0,0x0E09FaBB73Bd3Ade0a17ECC321fD13a19e81cE82,0xbb4CdB9CBd36B01bD1cBaEBF2De08d9173bc095c
2,0x58F876857a02D6762E0101bb5C46A8c1ED44Dc16,0xbb4CdB9CBd36B01bD1cBaEBF2De08d9173bc095c,0xe9e7CEA3DedcA5984780Bafc599bD69ADd087D56
3,0x28415ff2C35b65B9E5c7de82126b4015ab9d031F,0x3EE2200Efb3400fAbB9AacF31297cBdD1d435D47,0xbb4CdB9CBd36B01bD1cBaEBF2De08d9173bc095c
4,0x168B273278F3A8d302De5E879aA30690B7E6c28f,0xAD6cAEb32CD2c308980a548bD0Bc5AA4306c6c18,0xbb4CdB9CBd36B01bD1cBaEBF2De08d9173bc095c


In [33]:
tokens_appearances = uniswap_pools["token0"].to_list()
tokens_appearances.extend(uniswap_pools["token1"].to_list())
count = pd.Series(tokens_appearances).value_counts()
print("Total token",count)
print("Total Pair", len(uniswap_pools))
# print(count)
top5 =  count[:5]
# cd.pie(top5["symbol"].values, top5["count"].values, "", "", True)
print(top5/len(uniswap_pools) *100)
print(sum(count[5:]))
print(uniswap_tokens[uniswap_tokens["token"].isin(top5.index)])

Total token 0xbb4CdB9CBd36B01bD1cBaEBF2De08d9173bc095c    1510774
0x55d398326f99059fF775485246999027B3197955     100482
0xe9e7CEA3DedcA5984780Bafc599bD69ADd087D56      40626
0x0E09FaBB73Bd3Ade0a17ECC321fD13a19e81cE82       4572
0x2170Ed0880ac9A755fd29B2688956BD959F933F8       2132
                                               ...   
0x912967BaFb542392D550Be897022ce4306Ca3B9F          1
0xB49E10f9ff1d525c361C8d48830d3408a52Fa312          1
0x019668c08B6ec05d950a5ffAa861842F0635b336          1
0x3d1C63a4d4ef32E0c8E9aAdf98bFa3a558F62319          1
0x7c10527aFa530d01872ea77f1eaCfB85F3cF48F1          1
Name: count, Length: 1615131, dtype: int64
Total Pair 1699099
0xbb4CdB9CBd36B01bD1cBaEBF2De08d9173bc095c    88.916184
0x55d398326f99059fF775485246999027B3197955     5.913840
0xe9e7CEA3DedcA5984780Bafc599bD69ADd087D56     2.391032
0x0E09FaBB73Bd3Ade0a17ECC321fD13a19e81cE82     0.269084
0x2170Ed0880ac9A755fd29B2688956BD959F933F8     0.125478
Name: count, dtype: float64
1739612
                

# Cluster analysis

In [34]:
uni_file_path = os.path.join(path.univ2_processed_path, "non_swap_simple_rp_scammer_group.csv")
uni_groups = pd.read_csv(uni_file_path)
print("Unique scammers:", len(uni_groups["scammer"].unique()))
# print("Unique Pools:", len(uni_groups["pool"].unique()))
uni_groups = uni_groups.groupby('group_id')["scammer"].count().reset_index(name='num_scammer').sort_values(['num_scammer'], ascending=False)
print(uni_groups.head())
print(uni_groups.describe())
uoc = uni_groups[uni_groups.num_scammer == 1]
u_one_scammer_group_rate = len(uoc) / len(uni_groups)
print("one_scammer_group_rate", u_one_scammer_group_rate)

Unique scammers: 145654
     group_id  num_scammer
8           9         4155
104       105          646
585       586          538
737       738          432
195       196          395
            group_id    num_scammer
count  109471.000000  109471.000000
mean    54736.000000       1.330526
std     31601.699996      13.305829
min         1.000000       1.000000
25%     27368.500000       1.000000
50%     54736.000000       1.000000
75%     82103.500000       1.000000
max    109471.000000    4155.000000
one_scammer_group_rate 0.9226553151062838


In [38]:
uni_groups

,group_id,num_scammer
8,9,4155
104,105,646
585,586,538
737,738,432
195,196,395
...,...,...
39707,39708,1
39705,39706,1
39703,39704,1
39702,39703,1


In [35]:
pan_file_path = os.path.join(path.panv2_processed_path, "non_swap_simple_rp_scammer_group.csv")
pan_groups = pd.read_csv(pan_file_path)
print("Unique scammers:", len(pan_groups["scammer"].unique()))
pan_groups = pan_groups.groupby('group_id')["scammer"].count().reset_index(name='num_scammer').sort_values(['num_scammer'], ascending=False)
print(pan_groups.head())
print(pan_groups.describe())
poc = pan_groups[pan_groups.num_scammer == 1]
p_one_scammer_group_rate = len(poc) / len(pan_groups)
print("one_scammer_group_rate", p_one_scammer_group_rate)

Unique scammers: 18346
      group_id  num_scammer
7            8          571
276        277           83
92          93           43
284        285           39
1084      1085           33
          group_id   num_scammer
count  14561.00000  14561.000000
mean    7281.00000      1.259941
std     4203.54297      4.890597
min        1.00000      1.000000
25%     3641.00000      1.000000
50%     7281.00000      1.000000
75%    10921.00000      1.000000
max    14561.00000    571.000000
one_scammer_group_rate 0.8826316873841082


In [36]:
pan_groups

,group_id,num_scammer
7,8,571
276,277,83
92,93,43
284,285,39
1084,1085,33
...,...,...
5471,5472,1
5472,5473,1
5473,5474,1
5474,5475,1


# DATA SET

In [37]:
full_uni_pool_file_path = os.path.join(path.univ2_processed_path, "1_pair_pool_labels.csv")
full_uni_pool = pd.read_csv(full_uni_pool_file_path)
print("Uniswap pools", len(full_uni_pool))
print(full_uni_pool.head())
print("Unique creator", len(full_uni_pool.creator.unique()))


FileNotFoundError: [Errno 2] No such file or directory: '/mnt/Storage/Data/Blockchain/DEX/uniswap/processed/1_pair_pool_labels.csv'

In [6]:
uni_pool_file_path = os.path.join(path.univ2_processed_path, "filtered_simple_rp_scammers.csv")
uni_pool = pd.read_csv(uni_pool_file_path)
print("Uniswap scammer pools", len(uni_pool))
print(uni_pool.head())
print("Unique scammer", len(uni_pool.scammer.unique()))

Uniswap scammer pools 196239
                                         pool  \
0  0x2F2399AfBcF979cA3D92a3158562C9A397b28D06   
1  0x0e2129c1bDbb0AfaA6Be1ed53650B267C0FB5556   
2  0x7f6fe5c9FF1F9D46B29B79537D9ce6337127DFE0   
3  0x083B344D961D4Ab3823ACceC60253ec6c4bdB511   
4  0x441d3A8eA2076ad7326E91f52704CBE00C911F73   

                                      scammer  
0  0xe2772341c3ca68f332ec8d68d13b5ebae3dd26d8  
1  0x6f9ff1f95bde025e3d5e4633304fa8c29ea8acf8  
2  0xc1ff218cb1355e6c7b25e8144252cc5ea3c08d9f  
3  0xae9943ffb0ea5959ddbf7bbe93d246c861d4c939  
4  0xcb114805b901f7a9c38d5675272ef26459a7d805  
Unique scammer 176750


In [18]:
pan_pool_file_path = os.path.join(path.panv2_processed_path, "filtered_simple_rp_scammers.csv")
pan_pool = pd.read_csv(pan_pool_file_path)
print("Uniswap scammer pools", len(pan_pool))
print(pan_pool.head())
print("Unique scammer", len(pan_pool.scammer.unique()))

Uniswap scammer pools 105300
                                         pool  \
0  0x45c0a88988f33123794DAb66c232c00e7C1DdCd5   
1  0x45c0a88988f33123794DAb66c232c00e7C1DdCd5   
2  0xC403605005e2C878914ee9FEAD840490a961782b   
3  0xaaBd4586Ed416E14EBe6321523883EC36E387a47   
4  0xaaBd4586Ed416E14EBe6321523883EC36E387a47   

                                      scammer  
0  0xecaace2c92afe113057c1bf66c78ccd08e20d249  
1  0x348ad0b3a43baca917f2f75cd98172371bf191df  
2  0x7fb0792ad88320711270f219510bc3be928293c2  
3  0xecaace2c92afe113057c1bf66c78ccd08e20d249  
4  0x348ad0b3a43baca917f2f75cd98172371bf191df  
Unique scammer 59741


In [21]:
uni_event_stats = pd.read_csv("data/univ2_events_stats.csv")
print(uni_event_stats.head())
print("Mint", uni_event_stats.Mint.sum())
print("Burn", uni_event_stats.Burn.sum())
print("Transfer", uni_event_stats.Transfer.sum())
print("Swap", uni_event_stats.Swap.sum())

                                 pool_address  Mint  Burn  Transfer  Swap
0  0xB4e16d0168e52d35CaCD2c6185b44281Ec28C9Dc  1000  1000      1000  1000
1  0x3139Ffc91B99aa94DA8A2dc13f1fC36F9BDc98eE     8     6        21    45
2  0x12EDE161c702D1494612d19f05992f43aa6A26FB    35    27       100   896
3  0xA478c2975Ab1Ea89e8196811F51A7B7Ade33eB11  1000  1000      1000  1000
4  0x07F068ca326a469Fc1d87d85d448990C8cBa7dF9    44    37       119  1000
Mint 2191810
Burn 1022373
Transfer 4632651
Swap 49894733


In [22]:
pan_event_stats = pd.read_csv("data/panv2_events_stats.csv")
print(pan_event_stats.head())
print("Mint", pan_event_stats.Mint.sum())
print("Burn", pan_event_stats.Burn.sum())
print("Transfer", pan_event_stats.Transfer.sum())
print("Swap", pan_event_stats.Swap.sum())

                                 pool_address  Mint  Burn  Transfer  Swap
0  0x804678fa97d91B974ec2af3c843270886528a9E6  1000  1000      1000  1000
1  0x0eD7e52944161450477ee417DE9Cd3a859b14fD0  1000  1000      1000  1000
2  0x58F876857a02D6762E0101bb5C46A8c1ED44Dc16  1000  1000      1000  1000
3  0x28415ff2C35b65B9E5c7de82126b4015ab9d031F  1000  1000      1000  1000
4  0x168B273278F3A8d302De5E879aA30690B7E6c28f  1000  1000      1000  1000
Mint 17318233
Burn 5169489
Transfer 38054335
Swap 154969903
